Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
from tqdm import tqdm

First reload the data we generated in `1_notmnist.ipynb`.

In [ ]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

# Logistic model with L2 regularization

In [ ]:
batch_size = 128
regs = np.linspace(0.000001, 0.01, num=50)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = []
  for reg in regs:
    loss.append(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +
                               reg * tf.nn.l2_loss(weights))

  # Optimizer.
  optimizers = []
  for l in loss:
    optimizers.append(tf.train.GradientDescentOptimizer(0.5).minimize(l))

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [ ]:
num_steps = 3001

valid_acc = []
test_acc = []
for i in range(len(loss)):
    
    print('Regularization: %.6f' % regs[i], end='')
    
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      #print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizers[i], loss[i], train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          pass
          #print("Minibatch loss at step %d: %f" % (step, l))
          #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          #print("Validation accuracy: %.1f%%" % accuracy(
          #  valid_prediction.eval(), valid_labels))
      print(" Valid accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      valid_acc.append(accuracy(valid_prediction.eval(), valid_labels))
      test_acc.append(accuracy(test_prediction.eval(), test_labels))

plt.plot(regs, valid_acc)
plt.ylabel('valid accuracy [%]')
plt.show()

best_idx = valid_acc.index(max(valid_acc))
print("Best regularization: %f (valid accuracy: %.1f %%)" % (regs[best_idx], valid_acc[best_idx]))

print("Test accuracy: %.1f%%" % test_acc[best_idx])
        

We see that for regularization strength of 0.002042 we get test accuracy improvement of ~2.5%

# Neural Network model with L2 regularization

In [ ]:
batch_size = 128
hidden_layer_size = 1024
regs = np.linspace(0.000001, 0.01, num=50)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset  = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels   = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset  = tf.constant(valid_dataset)
  tf_test_dataset   = tf.constant(test_dataset)
  
  # Variables.
    
  # input layer
  weights1  = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1   = tf.Variable(tf.zeros([hidden_layer_size]))

  # hidden layer
  weights2  = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2   = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  l1_out    = tf.matmul(tf_train_dataset, weights1) + biases1      
  logits    = tf.matmul(tf.nn.relu(l1_out), weights2) + biases2
  loss = []
  for reg in regs:
    loss.append(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) +
                reg * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizers = []
  for l in loss:
    optimizers.append(tf.train.GradientDescentOptimizer(0.5).minimize(l))
  
  # Predictions for the training, validation, and test data.
  train_prediction  = tf.nn.softmax(logits)
  valid_prediction  = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction   = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [ ]:
num_steps = 3001

valid_acc = []
test_acc = []
for i in range(len(loss)):
    
    print('Regularization: %.6f' % regs[i], end='')
    
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      #print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizers[i], loss[i], train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          pass
          #print("Minibatch loss at step %d: %f" % (step, l))
          #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          #print("Validation accuracy: %.1f%%" % accuracy(
          #  valid_prediction.eval(), valid_labels))
      print(" Valid accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      valid_acc.append(accuracy(valid_prediction.eval(), valid_labels))
      test_acc.append(accuracy(test_prediction.eval(), test_labels))

plt.plot(regs, valid_acc)
plt.ylabel('valid accuracy [%]')
plt.show()

best_idx = valid_acc.index(max(valid_acc))
print("Best regularization: %f (valid accuracy: %.1f %%)" % (regs[best_idx], valid_acc[best_idx]))

print("Test accuracy: %.1f%%" % test_acc[best_idx])

We see that for regularization strength of 0.001429 we get test accuracy improvement of ~4%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
batch_size = 128
hidden_layer_size = 1024
reg = 0.001429
batch_num = 5

train_dataset_small = train_dataset[: batch_num * batch_size]
train_labels_small = train_labels[: batch_num * batch_size]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset  = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels   = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset  = tf.constant(valid_dataset)
  tf_test_dataset   = tf.constant(test_dataset)
  
  # Variables.
    
  # input layer
  weights1  = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1   = tf.Variable(tf.zeros([hidden_layer_size]))

  # hidden layer
  weights2  = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2   = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  l1_out    = tf.matmul(tf_train_dataset, weights1) + biases1      
  logits    = tf.matmul(tf.nn.relu(l1_out), weights2) + biases2
  loss      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss     += reg * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction  = tf.nn.softmax(logits)
  valid_prediction  = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction   = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  #print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print(" Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


We see that when the training is done with only a few batches (5), the accuracy is much lower than before ~10% less

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
hidden_layer_size = 1024
reg = 0.001429
batch_num = 5
dropout = 0.5

train_dataset_small = train_dataset[: batch_num * batch_size]
train_labels_small = train_labels[: batch_num * batch_size]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset  = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels   = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset  = tf.constant(valid_dataset)
  tf_test_dataset   = tf.constant(test_dataset)
  
  # Variables.
    
  # input layer
  weights1  = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1   = tf.Variable(tf.zeros([hidden_layer_size]))

  # hidden layer
  weights2  = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2   = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1        = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits    = tf.matmul(tf.nn.dropout(h1, dropout), weights2) + biases2
  loss      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss     += reg * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction  = tf.nn.softmax(logits)
  valid_prediction  = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction   = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  #print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print(" Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Dropout mechanism improves test accuracy by ~2%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
batch_size = 256
hidden_layer1 = 1024
hidden_layer2 = 256
num_steps = 20001
#reg = 0.001429
reg = 1e-4
dropout = 1.0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset  = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels   = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset  = tf.constant(valid_dataset)
  tf_test_dataset   = tf.constant(test_dataset)
  
  # Variables.
    
  # input layer
  weights1  = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1],
                                              stddev=np.sqrt(2.0/(image_size * image_size + hidden_layer1))))
  biases1   = tf.Variable(tf.zeros([hidden_layer1]))

  # hidden layer 1
  weights2  = tf.Variable(tf.truncated_normal([hidden_layer1, hidden_layer2],
                                              stddev=np.sqrt(2.0/(hidden_layer1 + hidden_layer2))))
  biases2   = tf.Variable(tf.zeros([hidden_layer2]))
    
  # hidden layer 2
  weights3  = tf.Variable(tf.truncated_normal([hidden_layer2, num_labels],
                                              stddev=np.sqrt(2.0/(hidden_layer2 + num_labels))))
  biases3   = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1        = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), dropout)
  h2        = tf.nn.dropout(tf.nn.relu(tf.matmul(h1, weights2) + biases2), dropout)      
  logits    = tf.matmul(h2, weights3) + biases3
  loss      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss     += reg * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.9)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
                                   tf.matmul(tf.nn.relu(
                                   tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), 
                                             weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
                                   tf.matmul(tf.nn.relu(
                                   tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), 
                                             weights3) + biases3)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  #print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print(" Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Test accuracy achieved: 96.3%.

The following improvements were done:

 1. An additinal hidden layer was added (256 nodes)
 
 2. Exponential learning rate decay was adopted
 
 3. Drop out was eliminated as it reduced accuracy
 
 4. Weights initialization was done with 2.0/n variance (n is the number of layer inputs + ouputs)
 
 5. Number of steps was increased to 20001
 
 6. Batch size was increased to 256
 
 7. Regularization strength was set to 0.0001